<a href="https://colab.research.google.com/github/tdogubo/population-data-analysis/blob/main/Tweets_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q tweepy emot

     |████████████████████████████████| 61 kB 16 kB/s 


In [2]:
import os
import nltk
import tweepy
import pandas as pd
import re
import pickle
from getpass import getpass as gp
from emot.emo_unicode import EMOJI_UNICODE, EMOJI_ALIAS_UNICODE, EMOTICONS_EMO

In [4]:
api_key = gp("Input apikey:: ")
api_key_secret = gp("Input apikey secret:: ")
access_token = gp("Input access token:: ")
access_token_secret = gp("Input access token secret:: ")


KeyboardInterrupt: ignored

In [5]:
auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token,access_token_secret)

api = tweepy.API(auth)

In [7]:
public_tweets = api.user_timeline(screen_name ="@elonmusk", count = 1000)
public_tweets[:3]

[Status(_api=<tweepy.api.API object at 0x7f660f555210>, _json={'created_at': 'Tue Feb 22 10:44:11 +0000 2022', 'id': 1496073325155692545, 'id_str': '1496073325155692545', 'text': '@GailAlfarATX @SpaceX @SirineAti @austinbarnard45 @OwenSparks_ @TacosandTeslas @EvaFoxU @DimaZeniuk @Grandpajoe42 @AstroJordy Yes', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'GailAlfarATX', 'name': 'Gail Alfar (Austin)', 'id': 1233929232432205827, 'id_str': '1233929232432205827', 'indices': [0, 13]}, {'screen_name': 'SpaceX', 'name': 'SpaceX', 'id': 34743251, 'id_str': '34743251', 'indices': [14, 21]}, {'screen_name': 'SirineAti', 'name': '💫Sirine💫', 'id': 1285872552766734346, 'id_str': '1285872552766734346', 'indices': [22, 32]}, {'screen_name': 'austinbarnard45', 'name': 'Austin Barnard🚀', 'id': 468706269, 'id_str': '468706269', 'indices': [33, 49]}, {'screen_name': 'OwenSparks_', 'name': 'Owen Sparks 🌎', 'id': 829829626180599808, 'id_str': '8298296261

In [8]:
tweets=[]
for tweet in public_tweets:
  tweets.append([tweet._json["created_at"],tweet._json["text"]])

print(tweets)

[['Tue Feb 22 10:44:11 +0000 2022', '@GailAlfarATX @SpaceX @SirineAti @austinbarnard45 @OwenSparks_ @TacosandTeslas @EvaFoxU @DimaZeniuk @Grandpajoe42 @AstroJordy Yes'], ['Tue Feb 22 10:26:34 +0000 2022', '@WholeMarsBlog V11 is 💥'], ['Tue Feb 22 10:21:48 +0000 2022', '@EricMagvas @westcoastbill Tuesday'], ['Tue Feb 22 10:21:06 +0000 2022', '@westcoastbill Yeah'], ['Tue Feb 22 10:18:25 +0000 2022', 'tau &gt; pi'], ['Tue Feb 22 10:10:05 +0000 2022', 'ℏ'], ['Tue Feb 22 09:59:00 +0000 2022', '@Teslarati @Writer_01001101 !!'], ['Tue Feb 22 09:18:58 +0000 2022', '@JohnnaCrider1 !'], ['Tue Feb 22 08:57:37 +0000 2022', '@Rainmaker1973 In Earth years'], ['Tue Feb 22 07:54:23 +0000 2022', '@PPathole @SpaceX @mayemusk It is taking longer than expected to make the upper stage not expendable (sigh)'], ['Tue Feb 22 06:45:15 +0000 2022', '@BBCScienceNews Instead of “research suggests”, I suggest “warlocks say”'], ['Tue Feb 22 06:42:47 +0000 2022', '@simongerman600 @waitbutwhy … a river'], ['Tue Feb 2

In [9]:
tweets_df = pd.DataFrame(tweets, columns =["created_at", "tweets"])
tweets_df

,created_at,tweets
0,Tue Feb 22 10:44:11 +0000 2022,@GailAlfarATX @SpaceX @SirineAti @austinbarnar...
1,Tue Feb 22 10:26:34 +0000 2022,@WholeMarsBlog V11 is 💥
2,Tue Feb 22 10:21:48 +0000 2022,@EricMagvas @westcoastbill Tuesday
3,Tue Feb 22 10:21:06 +0000 2022,@westcoastbill Yeah
4,Tue Feb 22 10:18:25 +0000 2022,tau &gt; pi
5,Tue Feb 22 10:10:05 +0000 2022,ℏ
6,Tue Feb 22 09:59:00 +0000 2022,@Teslarati @Writer_01001101 !!
7,Tue Feb 22 09:18:58 +0000 2022,@JohnnaCrider1 !
8,Tue Feb 22 08:57:37 +0000 2022,@Rainmaker1973 In Earth years
9,Tue Feb 22 07:54:23 +0000 2022,@PPathole @SpaceX @mayemusk It is taking longe...


In [10]:
import re
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F" # emoticons
                           u"\U0001F300-\U0001F5FF" # symbols & pictographs
                           u"\U0001F680-\U0001F6FF" # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF" # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    
    return emoji_pattern.sub(r'', string)
def convert_emojis(text):
      for emot in EMOJI_UNICODE:
          if EMOJI_UNICODE[emot] in text:
            text= text.replace(EMOJI_UNICODE[emot] , emot.replace(":","").replace("_"," "))
      return text
# Example
tweets_df['tweets_without_emojis']= tweets_df['tweets'].apply(lambda x: convert_emojis(x))
tweets_df
# text1 = "Hilarious 😂. The feeling of making a sale 😎, The feeling of actually fulfilling orders 😒"
# convert_emojis(text1)

,created_at,tweets,tweets_without_emojis
0,Tue Feb 22 10:44:11 +0000 2022,@GailAlfarATX @SpaceX @SirineAti @austinbarnar...,@GailAlfarATX @SpaceX @SirineAti @austinbarnar...
1,Tue Feb 22 10:26:34 +0000 2022,@WholeMarsBlog V11 is 💥,@WholeMarsBlog V11 is collision
2,Tue Feb 22 10:21:48 +0000 2022,@EricMagvas @westcoastbill Tuesday,@EricMagvas @westcoastbill Tuesday
3,Tue Feb 22 10:21:06 +0000 2022,@westcoastbill Yeah,@westcoastbill Yeah
4,Tue Feb 22 10:18:25 +0000 2022,tau &gt; pi,tau &gt; pi
5,Tue Feb 22 10:10:05 +0000 2022,ℏ,ℏ
6,Tue Feb 22 09:59:00 +0000 2022,@Teslarati @Writer_01001101 !!,@Teslarati @Writer_01001101 !!
7,Tue Feb 22 09:18:58 +0000 2022,@JohnnaCrider1 !,@JohnnaCrider1 !
8,Tue Feb 22 08:57:37 +0000 2022,@Rainmaker1973 In Earth years,@Rainmaker1973 In Earth years
9,Tue Feb 22 07:54:23 +0000 2022,@PPathole @SpaceX @mayemusk It is taking longe...,@PPathole @SpaceX @mayemusk It is taking longe...


In [11]:
import string
string.punctuation
#defining the function to remove punctuation

def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree
#storing the puntuation free text
tweets_df['clean_tweets']= tweets_df['tweets_without_emojis'].apply(lambda x:remove_punctuation(x))
tweets_df.head()

,created_at,tweets,tweets_without_emojis,clean_tweets
0,Tue Feb 22 10:44:11 +0000 2022,@GailAlfarATX @SpaceX @SirineAti @austinbarnar...,@GailAlfarATX @SpaceX @SirineAti @austinbarnar...,GailAlfarATX SpaceX SirineAti austinbarnard45 ...
1,Tue Feb 22 10:26:34 +0000 2022,@WholeMarsBlog V11 is 💥,@WholeMarsBlog V11 is collision,WholeMarsBlog V11 is collision
2,Tue Feb 22 10:21:48 +0000 2022,@EricMagvas @westcoastbill Tuesday,@EricMagvas @westcoastbill Tuesday,EricMagvas westcoastbill Tuesday
3,Tue Feb 22 10:21:06 +0000 2022,@westcoastbill Yeah,@westcoastbill Yeah,westcoastbill Yeah
4,Tue Feb 22 10:18:25 +0000 2022,tau &gt; pi,tau &gt; pi,tau gt pi


In [12]:
tweets_df['tweets_lower']= tweets_df['clean_tweets'].apply(lambda x: x.lower())
tweets_df

,created_at,tweets,tweets_without_emojis,clean_tweets,tweets_lower
0,Tue Feb 22 10:44:11 +0000 2022,@GailAlfarATX @SpaceX @SirineAti @austinbarnar...,@GailAlfarATX @SpaceX @SirineAti @austinbarnar...,GailAlfarATX SpaceX SirineAti austinbarnard45 ...,gailalfaratx spacex sirineati austinbarnard45 ...
1,Tue Feb 22 10:26:34 +0000 2022,@WholeMarsBlog V11 is 💥,@WholeMarsBlog V11 is collision,WholeMarsBlog V11 is collision,wholemarsblog v11 is collision
2,Tue Feb 22 10:21:48 +0000 2022,@EricMagvas @westcoastbill Tuesday,@EricMagvas @westcoastbill Tuesday,EricMagvas westcoastbill Tuesday,ericmagvas westcoastbill tuesday
3,Tue Feb 22 10:21:06 +0000 2022,@westcoastbill Yeah,@westcoastbill Yeah,westcoastbill Yeah,westcoastbill yeah
4,Tue Feb 22 10:18:25 +0000 2022,tau &gt; pi,tau &gt; pi,tau gt pi,tau gt pi
5,Tue Feb 22 10:10:05 +0000 2022,ℏ,ℏ,ℏ,ℏ
6,Tue Feb 22 09:59:00 +0000 2022,@Teslarati @Writer_01001101 !!,@Teslarati @Writer_01001101 !!,Teslarati Writer01001101,teslarati writer01001101
7,Tue Feb 22 09:18:58 +0000 2022,@JohnnaCrider1 !,@JohnnaCrider1 !,JohnnaCrider1,johnnacrider1
8,Tue Feb 22 08:57:37 +0000 2022,@Rainmaker1973 In Earth years,@Rainmaker1973 In Earth years,Rainmaker1973 In Earth years,rainmaker1973 in earth years
9,Tue Feb 22 07:54:23 +0000 2022,@PPathole @SpaceX @mayemusk It is taking longe...,@PPathole @SpaceX @mayemusk It is taking longe...,PPathole SpaceX mayemusk It is taking longer t...,ppathole spacex mayemusk it is taking longer t...


In [13]:
#defining function for tokenization
import re
def tokenization(text):
    tokens = re.split('W+',text)
    return tokens
#applying function to the column
tweets_df['tweets_tokenied']= tweets_df['tweets_lower'].apply(lambda x: tokenization(x))
tweets_df[:5]

,created_at,tweets,tweets_without_emojis,clean_tweets,tweets_lower,tweets_tokenied
0,Tue Feb 22 10:44:11 +0000 2022,@GailAlfarATX @SpaceX @SirineAti @austinbarnar...,@GailAlfarATX @SpaceX @SirineAti @austinbarnar...,GailAlfarATX SpaceX SirineAti austinbarnard45 ...,gailalfaratx spacex sirineati austinbarnard45 ...,[gailalfaratx spacex sirineati austinbarnard45...
1,Tue Feb 22 10:26:34 +0000 2022,@WholeMarsBlog V11 is 💥,@WholeMarsBlog V11 is collision,WholeMarsBlog V11 is collision,wholemarsblog v11 is collision,[wholemarsblog v11 is collision]
2,Tue Feb 22 10:21:48 +0000 2022,@EricMagvas @westcoastbill Tuesday,@EricMagvas @westcoastbill Tuesday,EricMagvas westcoastbill Tuesday,ericmagvas westcoastbill tuesday,[ericmagvas westcoastbill tuesday]
3,Tue Feb 22 10:21:06 +0000 2022,@westcoastbill Yeah,@westcoastbill Yeah,westcoastbill Yeah,westcoastbill yeah,[westcoastbill yeah]
4,Tue Feb 22 10:18:25 +0000 2022,tau &gt; pi,tau &gt; pi,tau gt pi,tau gt pi,[tau gt pi]


In [14]:
#Stop words present in the library
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
stopwords[0:10]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [15]:
def remove_stopwords(text):
    output= [i for i in text if i not in stopwords]
    return output

  #applying the function
tweets_df['tweets_no_stopwords']= tweets_df['tweets_tokenied'].apply(lambda x:remove_stopwords(x))
tweets_df[:5]

,created_at,tweets,tweets_without_emojis,clean_tweets,tweets_lower,tweets_tokenied,tweets_no_stopwords
0,Tue Feb 22 10:44:11 +0000 2022,@GailAlfarATX @SpaceX @SirineAti @austinbarnar...,@GailAlfarATX @SpaceX @SirineAti @austinbarnar...,GailAlfarATX SpaceX SirineAti austinbarnard45 ...,gailalfaratx spacex sirineati austinbarnard45 ...,[gailalfaratx spacex sirineati austinbarnard45...,[gailalfaratx spacex sirineati austinbarnard45...
1,Tue Feb 22 10:26:34 +0000 2022,@WholeMarsBlog V11 is 💥,@WholeMarsBlog V11 is collision,WholeMarsBlog V11 is collision,wholemarsblog v11 is collision,[wholemarsblog v11 is collision],[wholemarsblog v11 is collision]
2,Tue Feb 22 10:21:48 +0000 2022,@EricMagvas @westcoastbill Tuesday,@EricMagvas @westcoastbill Tuesday,EricMagvas westcoastbill Tuesday,ericmagvas westcoastbill tuesday,[ericmagvas westcoastbill tuesday],[ericmagvas westcoastbill tuesday]
3,Tue Feb 22 10:21:06 +0000 2022,@westcoastbill Yeah,@westcoastbill Yeah,westcoastbill Yeah,westcoastbill yeah,[westcoastbill yeah],[westcoastbill yeah]
4,Tue Feb 22 10:18:25 +0000 2022,tau &gt; pi,tau &gt; pi,tau gt pi,tau gt pi,[tau gt pi],[tau gt pi]


In [16]:
#importing the Stemming function from nltk library
from nltk.stem.porter import PorterStemmer
#defining the object for stemming
porter_stemmer = PorterStemmer()
#defining a function for stemming
def stemming(text):
  stem_text = [porter_stemmer.stem(word) for word in text]
  return stem_text

tweets_df['tweets_stemmed']=tweets_df['tweets_no_stopwords'].apply(lambda x: stemming(x))
tweets_df[:5]

,created_at,tweets,tweets_without_emojis,clean_tweets,tweets_lower,tweets_tokenied,tweets_no_stopwords,tweets_stemmed
0,Tue Feb 22 10:44:11 +0000 2022,@GailAlfarATX @SpaceX @SirineAti @austinbarnar...,@GailAlfarATX @SpaceX @SirineAti @austinbarnar...,GailAlfarATX SpaceX SirineAti austinbarnard45 ...,gailalfaratx spacex sirineati austinbarnard45 ...,[gailalfaratx spacex sirineati austinbarnard45...,[gailalfaratx spacex sirineati austinbarnard45...,[gailalfaratx spacex sirineati austinbarnard45...
1,Tue Feb 22 10:26:34 +0000 2022,@WholeMarsBlog V11 is 💥,@WholeMarsBlog V11 is collision,WholeMarsBlog V11 is collision,wholemarsblog v11 is collision,[wholemarsblog v11 is collision],[wholemarsblog v11 is collision],[wholemarsblog v11 is collis]
2,Tue Feb 22 10:21:48 +0000 2022,@EricMagvas @westcoastbill Tuesday,@EricMagvas @westcoastbill Tuesday,EricMagvas westcoastbill Tuesday,ericmagvas westcoastbill tuesday,[ericmagvas westcoastbill tuesday],[ericmagvas westcoastbill tuesday],[ericmagvas westcoastbill tuesday]
3,Tue Feb 22 10:21:06 +0000 2022,@westcoastbill Yeah,@westcoastbill Yeah,westcoastbill Yeah,westcoastbill yeah,[westcoastbill yeah],[westcoastbill yeah],[westcoastbill yeah]
4,Tue Feb 22 10:18:25 +0000 2022,tau &gt; pi,tau &gt; pi,tau gt pi,tau gt pi,[tau gt pi],[tau gt pi],[tau gt pi]


In [17]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
#defining the object for Lemmatization
wordnet_lemmatizer = WordNetLemmatizer()
#defining the function for lemmatization
def lemmatizer(text):
  lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
  return lemm_text
tweets_df['tweets_lemmatized']=tweets_df['tweets_no_stopwords'].apply(lambda x:lemmatizer(x))
tweets_df[:5]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


,created_at,tweets,tweets_without_emojis,clean_tweets,tweets_lower,tweets_tokenied,tweets_no_stopwords,tweets_stemmed,tweets_lemmatized
0,Tue Feb 22 10:44:11 +0000 2022,@GailAlfarATX @SpaceX @SirineAti @austinbarnar...,@GailAlfarATX @SpaceX @SirineAti @austinbarnar...,GailAlfarATX SpaceX SirineAti austinbarnard45 ...,gailalfaratx spacex sirineati austinbarnard45 ...,[gailalfaratx spacex sirineati austinbarnard45...,[gailalfaratx spacex sirineati austinbarnard45...,[gailalfaratx spacex sirineati austinbarnard45...,[gailalfaratx spacex sirineati austinbarnard45...
1,Tue Feb 22 10:26:34 +0000 2022,@WholeMarsBlog V11 is 💥,@WholeMarsBlog V11 is collision,WholeMarsBlog V11 is collision,wholemarsblog v11 is collision,[wholemarsblog v11 is collision],[wholemarsblog v11 is collision],[wholemarsblog v11 is collis],[wholemarsblog v11 is collision]
2,Tue Feb 22 10:21:48 +0000 2022,@EricMagvas @westcoastbill Tuesday,@EricMagvas @westcoastbill Tuesday,EricMagvas westcoastbill Tuesday,ericmagvas westcoastbill tuesday,[ericmagvas westcoastbill tuesday],[ericmagvas westcoastbill tuesday],[ericmagvas westcoastbill tuesday],[ericmagvas westcoastbill tuesday]
3,Tue Feb 22 10:21:06 +0000 2022,@westcoastbill Yeah,@westcoastbill Yeah,westcoastbill Yeah,westcoastbill yeah,[westcoastbill yeah],[westcoastbill yeah],[westcoastbill yeah],[westcoastbill yeah]
4,Tue Feb 22 10:18:25 +0000 2022,tau &gt; pi,tau &gt; pi,tau gt pi,tau gt pi,[tau gt pi],[tau gt pi],[tau gt pi],[tau gt pi]


In [18]:
from textblob import TextBlob
# function to calculate subjectivity
# def getSubjectivity(review):
#   return TextBlob(review).sentiment.subjectivity
# function to calculate polarity
def getPolarity(review):
  stem_text = [porter_stemmer.stem(word) for word in review]
  # return stem_text
  return TextBlob(stem_text[0]).sentiment.polarity

# function to analyze the reviews
def analysis(score):
    if score < 0:
        return 'Negative'
    elif score == 0:
        return 'Neutral'
    else:
        return 'Positive'

In [19]:
sentiment_df = pd.DataFrame(tweets_df[['tweets', 'tweets_lemmatized']])
# fin_data['Subjectivity'] = fin_data['Lemma'].apply(getSubjectivity) 
sentiment_df['Polarity'] = sentiment_df['tweets_lemmatized'].apply(lambda x:getPolarity(x)) 
sentiment_df['Analysis'] = sentiment_df['Polarity'].apply(analysis)
sentiment_df

,tweets,tweets_lemmatized,Polarity,Analysis
0,@GailAlfarATX @SpaceX @SirineAti @austinbarnar...,[gailalfaratx spacex sirineati austinbarnard45...,0.00,Neutral
1,@WholeMarsBlog V11 is 💥,[wholemarsblog v11 is collision],0.00,Neutral
2,@EricMagvas @westcoastbill Tuesday,[ericmagvas westcoastbill tuesday],0.00,Neutral
3,@westcoastbill Yeah,[westcoastbill yeah],0.00,Neutral
4,tau &gt; pi,[tau gt pi],0.00,Neutral
5,ℏ,[ℏ],0.00,Neutral
6,@Teslarati @Writer_01001101 !!,[teslarati writer01001101 ],0.00,Neutral
7,@JohnnaCrider1 !,[johnnacrider1 ],0.00,Neutral
8,@Rainmaker1973 In Earth years,[rainmaker1973 in earth years],0.00,Neutral
9,@PPathole @SpaceX @mayemusk It is taking longe...,[ppathole spacex mayemusk it is taking longer ...,-0.05,Negative


In [20]:
export_df = pd.DataFrame(sentiment_df[['tweets', 'Analysis']])
export_df[:7]

,tweets,Analysis
0,@GailAlfarATX @SpaceX @SirineAti @austinbarnar...,Neutral
1,@WholeMarsBlog V11 is 💥,Neutral
2,@EricMagvas @westcoastbill Tuesday,Neutral
3,@westcoastbill Yeah,Neutral
4,tau &gt; pi,Neutral
5,ℏ,Neutral
6,@Teslarati @Writer_01001101 !!,Neutral


In [22]:
export_df.to_csv("tweets.csv")